In [1]:
from pyspark.sql import SparkSession 

In [2]:
spark = SparkSession.builder.appName('project').getOrCreate()

In [3]:
df = spark.read.csv('walmart_stock.csv' , inferSchema= True , header= True)

In [4]:
# Task 1 

df.show(3)

+----------+------------------+---------+---------+------------------+--------+------------------+
|      Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+----------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
+----------+------------------+---------+---------+------------------+--------+------------------+
only showing top 3 rows



In [5]:
# Task 2 
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

In [6]:
# Task 4 

df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [7]:
for i in df.head(5):
    print(i)

Row(Date='2012-01-03', Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996)
Row(Date='2012-01-04', Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475)
Row(Date='2012-01-05', Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539)
Row(Date='2012-01-06', Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922)
Row(Date='2012-01-09', Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)


In [44]:
df.describe().show(3)

+-------+----+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|Date|             Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+----+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|1258|             1258|             1258|             1258|             1258|             1258|             1258|
|   mean|null|72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|null| 6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
+-------+----+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
only showing top 3 rows



In [9]:
# std , mean  = needs to be formated to only two decimels 

from pyspark.sql.functions import (stddev , mean , format_number , min , max)

In [10]:
df.describe().printSchema()

root
 |-- summary: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



In [13]:
df.describe()

DataFrame[summary: string, Date: string, Open: string, High: string, Low: string, Close: string, Volume: string, Adj Close: string]

In [30]:
minimize_describe = df.describe()
minimize_describe.select(minimize_describe['summary'] ,
                        format_number(minimize_describe['Open'].cast('float'),2).alias('Open'),
                        format_number(minimize_describe['High'].cast('float'),2).alias('High'),
                        format_number(minimize_describe['Low'].cast('float'),2).alias('Low'),
                        format_number(minimize_describe['Close'].cast('float'),2).alias('Close'),
                        minimize_describe['Volume'].cast('int').alias('Volume')
                        ).show()

+-------+--------+--------+--------+--------+--------+
|summary|    Open|    High|     Low|   Close|  Volume|
+-------+--------+--------+--------+--------+--------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|    1258|
|   mean|   72.36|   72.84|   71.92|   72.39| 8222093|
| stddev|    6.77|    6.77|    6.74|    6.76| 4519780|
|    min|   56.39|   57.06|   56.30|   56.42| 2094900|
|    max|   90.80|   90.97|   89.25|   90.47|80898100|
+-------+--------+--------+--------+--------+--------+



In [43]:
df.withColumn('HV Ratio' , (df['High'] / df['Volume']))\
    .select('HV Ratio')\
    .show(2)

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
+--------------------+
only showing top 2 rows



In [50]:
from pyspark.sql.functions import (max , mean , min , corr)

In [40]:
df.orderBy(df['High'].desc()).head(1)

[Row(Date='2015-01-13', Open=90.800003, High=90.970001, Low=88.93, Close=89.309998, Volume=8215400, Adj Close=83.825448)]

In [46]:
df.select(mean('Close')).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



In [48]:
df.select(  max('Volume') ,
            min('Volume')).show()

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+



In [51]:
df.select(corr('High' , 'Volume')).show()

+-------------------+
| corr(High, Volume)|
+-------------------+
|-0.3384326061737161|
+-------------------+

